In [1]:
import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.text as kpt
import keras
import re 
import numpy, time
import random
import pickle
from keras.models import load_model, model_from_json
import random

height has been deprecated.



Using TensorFlow backend.


In [2]:
def transform_data_for_tokenizer(recipe_item):
    original = recipe_item.lower()
    original = re.sub(r'\s*(\d+|[./+*-])', '', original)
    original_split = original.split(" ")
    remove_list = list(set(original).symmetric_difference(units_of_food_recipe))

    keep_list = []
    for i in original_split:
        if i not in food_size and i not in units_of_food_recipe:
            keep_list.append(i)
    
    original_split = list(filter(None, keep_list))
    new = " ".join(original_split)
    return new

In [3]:
units_of_food_recipe = ["cups", "cup", "c.", "c", 
                        "fl. oz.", "fl oz", "fluid ounce", "fluid ounces",
                        "gal", "gal.", "gallon", "gallons",
                        "oz", "oz.", "ounce", "ounces", "ouncs",
                        "pt", "pt.", "pint", "pints",
                        "lb", "lb.", "pound", "pounds",
                        "qt", "qt.", "qts", "qts.", "quart", "quarts",
                        "tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs",
                        "tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons",
                        "g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes",
                        "kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes",
                        "l", "l.", "liter", "liters", "litre", "litres",
                        "mg", "mg.", "milligram", "milligrams", "milligramme", "milligrammes",
                        "ml", "ml.", "milliliter", "milliliters", "millilitre", "millilitres",
                        "pinch", "pinches", "dash", "dashes", "touch", "touches", "handful", "handfuls",
                        "stick", "sticks",
                        "cans", "can",
                        "to taste",
                        "scoop", "scoops",
                        "dollop", "dollops",
                        "sprig", "sprigs",
                       "recipe",
                       "garnish", "garnished",
                       "sprinkle", "spinkled",
                       "slices",
                       "serving", "servings",
                       "ribs", "rib", "stalk",
                       "inch", "inches", "in.",
                       "drizzle", "drizzled",
                       "to taste"]

food_size = ["small", "medium",
              "large", "about"]

food_format = ["slice", "sliced",
              "dice", "diced",
              "peel", "peeled",
              "chop", "chopped",
              "julienne", "julienned",
              "trim", "trimmed",
              "sift", "sifted",
              "freshly",
              "fine", "finely",
              "drain", "drained",
              "super thinly", "thin", "thinly",
              "grate", "grated",
              "seeded",
              "rinse", "rinsed",
              "smash", "smashed",
              "membrane", "removed",
              "cleaned",
              "mince", "minced",
              "crush", "crushed",
              "about", "around",
              '<hr>']


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
# nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df.csv", index_col=0)
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df_MAURA.csv", index_col=0)

master_df = pd.read_csv("../../data/nutrient/compiled/nutrition_master_df.csv")

with open('../../data/recipe/recipe_all.json') as f:
    recipe_all = json.load(f)
    
with open('/root/w210_capstone/data/models/NDB_tag_unique_unique_dict.json') as f:
        NDB_tag_unique_unique_dict = json.load(f)

# loading
with open('/root/w210_capstone/data/models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
# load json and create model
json_file = open('/root/w210_capstone/data/models/model_simple_nn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/root/w210_capstone/data/models/model_simple_nn_WEIGHTS.h5")
print("Loaded model from disk")

ambiguous = pd.read_csv("/root/w210_capstone/data/recipe/ambiguous_conversion.csv")

Loaded model from disk


In [8]:
random.seed(time.time())
ids = np.unique(list(recipe_all.keys()))
random.shuffle(ids)

original_text_list = []
NDB_ID_list = []
Description_list = []

for recipe_id in ids[:5]:
    print('\n')
    print(recipe_id)
    
    for ingredient in recipe_all[recipe_id]['ingredients']:
        print(ingredient)
        ingredient_filtered = transform_data_for_tokenizer(ingredient)
        tokens = tokenizer.texts_to_matrix([ingredient_filtered])
        n = 3
        prediction = loaded_model.predict(np.array(tokens))

        index_top_3_list =  loaded_model.predict(np.array(tokens)).argsort()[0][-n:][::-1]
        NDB_top_3_list = []
        percentage_top_3_list = []
        for i in index_top_3_list:   
            for key in NDB_tag_unique_unique_dict.keys():        
                if NDB_tag_unique_unique_dict[key] == i:
                    NDB_top_3_list.append(key)
                    percentage_top_3_list.append(prediction[0][i])

        itr = 0
        while itr < 3:            
            print('\t', NDB_top_3_list[itr], round(percentage_top_3_list[itr], 4), master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[itr].strip('"'))]['Description'].get_values()[0])
            itr += 1
        
        print('\n')
        
        raw_input_return = input()
        
        if raw_input_return == 'exit':
            break
        
        else:
            if raw_input_return:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(raw_input_return)
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(raw_input_return.strip('"'))]['Description'].get_values()[0])
            else:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(NDB_top_3_list[0])
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[0].strip('"'))]['Description'].get_values()[0])
                

temp_df = pd.DataFrame([original_text_list, NDB_ID_list, Description_list]).transpose()
temp_df.columns = nlp_df.columns
temp_df = temp_df.dropna()
nlp_df_new = pd.concat([nlp_df, temp_df])
nlp_df_new.to_csv("../../data/nutrient/compiled/train_nn_df_MAURA.csv")
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df_MAURA.csv", index_col=0)



RECIPE_75039
2-3 teaspoons Dijon mustard
	 "02046" 0.9971 mustard, prepared, yellow
	 "02024" 0.0025 spices, mustard seed, ground
	 "11935" 0.0 catsup



salt & pepper to taste
	 "02047" 0.996 salt, table
	 "02030" 0.0013 spices, pepper, black
	 "19296" 0.0007 honey



1/4 cup red wine vinegar
	 "02053" 0.4674 vinegar, distilled
	 "02069" 0.1964 vinegar, balsamic
	 "04135" 0.1631 salad dressing, home recipe, vinegar and oil



3/4 cup extra virgin olive oil
	 "04708" 0.9927 mayonnaise, reduced fat, with olive oil
	 "44005" 0.0059 oil, corn, peanut, and olive
	 "09042" 0.0003 blackberries, raw





RECIPE_66540
2 tablespoons extra-virgin olive oil
	 "44005" 0.7884 oil, corn, peanut, and olive
	 "04708" 0.2074 mayonnaise, reduced fat, with olive oil
	 "16087" 0.0022 peanuts, all types, raw




1/4 cup chopped onion
	 "11282" 0.9973 onions, raw
	 "11291" 0.0015 onions, spring or scallions (includes tops and bulb), raw
	 "11294" 0.0006 onions, sweet, raw


1/4 cup chopped Italian sausage


1/2 cup maple syrup
	 "19353" 0.9979 syrups, maple
	 "09043" 0.0002 blackberry juice, canned
	 "19912" 0.0002 sweetener, syrup, agave



1 teaspoon maple extract
	 "19353" 0.7299 syrups, maple
	 "02050" 0.1389 vanilla extract
	 "19202" 0.0221 puddings, vanilla, dry mix, instant



8 ounces bacon cooked until crisp and roughly diced
	 "10860" 0.9458 pork, cured, bacon, cooked, baked
	 "10123" 0.0305 pork, cured, bacon, unprepared
	 "11205" 0.0032 cucumber, with peel, raw



8 ounces sour cream
	 "01056" 0.9515 cream, sour, cultured
	 "01180" 0.0268 sour cream, fat free
	 "01053" 0.0068 cream, fluid, heavy whipping



1/4 cup sugar
	 "19335" 0.9831 sugars, granulated
	 "28293" 0.0088 cookie, butter or sugar, with chocolate icing or filling
	 "19336" 0.0032 sugars, powdered



1 cup maple syrup
	 "19353" 0.9979 syrups, maple
	 "09043" 0.0002 blackberry juice, canned
	 "19912" 0.0002 sweetener, syrup, agave



1 tablespoon vanilla extract
	 "02050" 0.9982 vanilla extract
	 "19293" 0.0004 


1 cup carrots diced cooked
	 "11124" 0.9915 carrots, raw
	 "11960" 0.0058 carrots, baby, raw
	 "11298" 0.0003 parsnips, raw



4 pounds potatoes (with skin) boiled
	 "11352" 0.6574 potatoes, flesh and skin, raw
	 "11507" 0.251 sweet potato, raw, unprepared (includes foods for usda's food distribution program)
	 "11362" 0.0233 potatoes, raw, skin



1/2 gallon milk
	 "01077" 0.8325 milk, whole, 3.25% milkfat, with added vitamin d
	 "14312" 0.0577 beverages, malted drink mix, natural, powder, prepared with whole milk
	 "01085" 0.0359 milk, nonfat, fluid, with added vitamin a and vitamin d (fat free or skim)



1/2 pound grated Parmesan cheese
	 "01033" 0.999 cheese, parmesan, hard
	 "01040" 0.0003 cheese, swiss
	 "01020" 0.0002 cheese, fontina



Seasonal vegetables for serving
	 "09042" 0.0737 blackberries, raw
	 "04679" 0.0391 oil, pam cooking spray, original
	 "01057" 0.0379 eggnog


"43312"


RECIPE_73094


RECIPE_8908
2 lb skinless boneless chicken thighs trimmed
	 "05062" 0.9603 c

In [6]:
len(nlp_df)

213